### 参考：
    https://www.cnblogs.com/qxh-beijing2016/p/12084150.html
    https://www.cnblogs.com/lee0oo0/p/4466090.html

#### フォルダ名を変更

In [ ]:
def rename(path):
    i=0
    name_list=['kaggle','tableau','pytorch']
    FileList = os.listdir(path)
    for files in FileList:
        oldDirPath = os.path.join(path, files)
        if os.path.isdir(oldDirPath):
            rename(oldDirPath)            
            newDirPath=os.path.join(path,name_list[i])
            os.rename(oldDirPath, newDirPath)
            print(files +' has changed as '+newDirPath)
            i += 1

In [ ]:
# !pip install deepdiff
# !pip install BeautifulSoup4
# !pip install lxml
# !pip install langid 
# !pip install mecab-python3
# !pip install nltk
# !pip install extract_msg

In [1]:
import os
from shutil import copy
from pathlib import Path
import email
import extract_msg
from bs4 import BeautifulSoup
# nltk.download('punkt')
# from deepdiff import DeepDiff

#### 各フォルダ内にて比較

In [2]:
kk_full=[k for k,w,t in os.walk('./A_full')]
ww_full=[w for k,w,t in os.walk('./A_full')]
tt_full=[t for k,w,t in os.walk('./A_full')]

kk_error=[k for k,w,t in os.walk('./B_error')]
ww_error=[w for k,w,t in os.walk('./B_error')]
tt_error=[t for k,w,t in os.walk('./B_error')]

In [3]:
kk_full

['./A_full',
 './A_full\\20210201',
 './A_full\\20210202',
 './A_full\\20210203',
 './A_full\\20210204',
 './A_full\\20210205',
 './A_full\\20210206',
 './A_full\\20210207',
 './A_full\\20210208']

In [4]:
file_name_full = [kk_full[i].split('/')[-1].split('\\')[-1] for i in range(1,len(kk_full))]
file_name_full
file_name_error = [kk_error[i].split('/')[-1].split('\\')[-1] for i in range(1,len(kk_error))]
file_name_error
file_name_error

['a', 'A1', 'a1', 'a10', 'a11', 'A19', 'a2', 'S']

### 並び替えルールを定義する
    chr(ord(c)+58):https://techacademy.jp/magazine/36090#1

In [5]:
from itertools import count
def rank_filename(value):
    if len(value) <= 1:
        for i in count():
            try:
                _ = chr(i)
            except ValueError:
                max_char_number = i-1
                break
        left,right = value.lower(),chr(max_char_number)    
    else:
        left,right = value[0],''.join(list(value)[1:])
        #A-Zのほうがa-zより小さいので、それを入れ替える
        left_after = ''
        for c in left:
            if 'a' <= c <= 'z':
                left_after += chr(ord('A')+(ord(c)-(ord('a')))+5)
            elif 'A' <= c <= 'Z':
                left_after += chr(ord('a')+(ord(c)-(ord('A')))+5)
            else:
                left_after += c
        left = left_after      
    return left,right

In [6]:
file_name_sort = sorted(file_name_error,key=rank_filename)
file_name_sort

['a1', 'a10', 'a11', 'a2', 'a', 'A1', 'A19', 'S']

In [7]:
### 参考用
ord('A'),ord('a')+(ord('C')-(ord('A'))-45),ord('S'),ord('a'),ord('s'),ord('-')

(65, 54, 83, 97, 115, 45)

### 並び替え方法②
    file_name_sort =sorted(file_name_old,key=lambda x:(''.join(chr(ord(c)+58) if 'A' <= c <= 'Z' 
                                             else c for c in x.split('-')[0]),(x+'-').split('-')[1] 
                                             or chr(1114111)))


In [8]:
folder_path_sorted = ['./B_error/'+i for i in file_name_sort]
folder_path_full = ['./A_full/'+i for i in file_name_full]
folder_path_sorted[1]

'./B_error/a10'

In [9]:
folder_path_sorted

['./B_error/a1',
 './B_error/a10',
 './B_error/a11',
 './B_error/a2',
 './B_error/a',
 './B_error/A1',
 './B_error/A19',
 './B_error/S']

In [10]:
def make_full_path(list_):
    full_path = []
    for i in list_:
        for v,k,m in os.walk(i):
            path_tmp = [os.path.join(v,x) for x in m]
            full_path.append(path_tmp)
    return full_path

In [11]:
full_path_sorted = make_full_path(folder_path_sorted)
full_path_all = make_full_path(folder_path_full)
full_path_sorted[7][0]

'./B_error/S\\全員に１レッスンプレゼント！仙台校移転記念♪トライアルキャンペーン開催！！.eml'

In [12]:
file_path_dict=dict(zip(folder_path_sorted,folder_path_full))
file_path_dict

{'./B_error/a1': './A_full/20210201',
 './B_error/a10': './A_full/20210202',
 './B_error/a11': './A_full/20210203',
 './B_error/a2': './A_full/20210204',
 './B_error/a': './A_full/20210205',
 './B_error/A1': './A_full/20210206',
 './B_error/A19': './A_full/20210207',
 './B_error/S': './A_full/20210208'}

### A.内容をもとに比較

#### eml & msg

In [14]:
def extract_text(list_):
    text = []
    file_id = []
    for i in list_:
        if i.split('.')[-1] == 'eml':
            file_id.append(i)
            with open(i,'r',encoding = 'utf-8-sig') as f:
                text_ = f.read()
                text.append(text_)
#                 print('extracted',i)
        elif i.split('.')[-1] == 'msg':
            file_id.append(i)
            subject = extract_msg.Message(i).subject
            date = extract_msg.Message(i).date
            body = extract_msg.Message(i).body
            text_ = subject+date+body
            text.append(text_)
#             print('extracted',i)
        elif i.split('.')[-1] == 'html':
            file_id.append(i)
            htmlfile = open(i, 'r', encoding='utf-8-sig')
            htmlhandle = htmlfile.read()
            soup = BeautifulSoup(htmlhandle, 'html.parser')
            text.append(soup.text)
#             print('extracted',i)
            
#         dict_ = dict(zip(file_id,text))
    return file_id,text         

##### 内容を比較し、idを取り出す

In [15]:
differ_ids=[]
for i in range(len(full_path_all)):
    file_id,text_sorted =extract_text(full_path_all[i])
    file_id_error,text_error =extract_text(full_path_sorted[i])
    dict_ = dict(zip(text_sorted,file_id))
    differ_id = []
    for m in text_sorted:
        if m not in text_error:
            differ_id.append(dict_[m])
    differ_ids.append(differ_id)

In [16]:
differ_ids[1]

['./A_full/20210202\\Your question of the week from Vocabulary.com.eml',
 './A_full/20210202\\東京2020大会：オリンピック＆パラリンピックの世界へ.eml']

#### 保存する

In [17]:
save_dir='./new'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

In [24]:
for i in differ_ids:
    for m in i:
#         print(m.split('/')[2].split('\\')[0])
        to_path = os.path.join(save_dir,m.split('/')[2].split('\\')[0])
        if not os.path.isdir(to_path):
            os.makedirs(to_path)
        copy(m,to_path)

### B.件名をもとに比較

In [19]:
folder_path_sorted = ['./B_error/'+i for i in file_name_sort]
folder_path_full = ['./A_full/'+i for i in file_name_full]
folder_path_sorted[1]

'./B_error/a10'

In [20]:
kk_error_sorted=[k for i in folder_path_sorted for k,w,t in os.walk(i)]
# ww_sorted=[w for i in folder_path_sorted for k,w,t in os.walk(i)]
tt_error_sorted=[t for i in folder_path_sorted for k,w,t in os.walk(i)]

kk_full=[k for k,w,t in os.walk('./A_full')]
ww_full=[w for k,w,t in os.walk('./A_full')]
tt_full=[t for k,w,t in os.walk('./A_full')]

In [21]:
kk_full

['./A_full',
 './A_full\\20210201',
 './A_full\\20210202',
 './A_full\\20210203',
 './A_full\\20210204',
 './A_full\\20210205',
 './A_full\\20210206',
 './A_full\\20210207',
 './A_full\\20210208']

In [22]:
tt_full

[[],
 ['Nintendo Switch『モンスターハンターライズ』の最新映像が公開。 _ トピックス _ Nintendo.html',
  'The savings are clear.eml',
  '[asjnews01 _199] AS-J 権利擁護支援ニュース (21_01_ 30 Sat).eml',
  '「僕もこんな本を作りたかった」編集者が嫉妬した２冊.eml',
  '【Nintendo Switch】期間限定セール・新作ほか、ソフトのご紹介.eml'],
 ["We're not seeing any activity from last week..eml",
  "We've got so much to catch up on!.eml",
  'Your question of the week from Vocabulary.com.eml',
  '東京2020大会：オリンピック＆パラリンピックの世界へ.eml'],
 ['21歳の学生がオーナー。ラベンダー色の店に、服も夢も詰め込んで\u3000「喫茶ジョー」 _ 朝日新聞デジタル＆w（アンド・ダブリュー）.html',
  'LAST CHANCE_ $75 annual plans end tonight!.eml',
  '[蝴蝶效应] 摘要.eml',
  'もっちもっちの弾力に、脈打つ熟成香。魅惑のブランデーあんぱん.eml',
  '新年のご挨拶.eml'],
 ['40歳、不妊治療をやめた。手にした“奇跡”に、さよならを.eml',
  'Communication trends of 2020.eml',
  '[asjnews01 _195] AS-J 権利擁護支援ニュース (20_12_ 20 Sun).eml',
  'ゴーストライター騒動から7年 新垣隆の「覚悟」.eml',
  'ソフトウェアを安全に保護するためのより良い方法.eml'],
 ['【スーパーマリオブラザーズ35周年】12月のニュースをお届けいたします.eml',
  'スカウト担当より求人のご案内_スズキ株式会社_データサイエンティスト_データエンジニア【リクルートエージェント】.eml',
  '宇賀なつみ、思わず息をのむ.eml',
  '東京2020大会：オリンピック＆パラリ

In [25]:
list_ = [[]]*8
for i in range(len(tt_error_sorted)):
    list_.append(list(set(tt_full[i+1])-set(tt_error_sorted[i])))

In [26]:
list_

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Nintendo Switch『モンスターハンターライズ』の最新映像が公開。 _ トピックス _ Nintendo.html',
  'The savings are clear.eml',
  '【Nintendo Switch】期間限定セール・新作ほか、ソフトのご紹介.eml'],
 ['Your question of the week from Vocabulary.com.eml',
  '東京2020大会：オリンピック＆パラリンピックの世界へ.eml'],
 ['新年のご挨拶.eml', 'もっちもっちの弾力に、脈打つ熟成香。魅惑のブランデーあんぱん.eml'],
 ['ソフトウェアを安全に保護するためのより良い方法.eml',
  'ゴーストライター騒動から7年 新垣隆の「覚悟」.eml',
  'Communication trends of 2020.eml'],
 ['東京2020大会：オリンピック＆パラリンピックの世界を知り尽くそう.eml', '宇賀なつみ、思わず息をのむ.eml'],
 ['深度学习之路（一）：用LSTM网络做时间序列数据预测 - 简书.html'],
 ['Our cool-productivity-killer newsletter.eml'],
 ['明智光秀、最後の夜は…….eml']]

In [27]:
len(list_[8:])

8

In [28]:
dict_path = dict(zip(kk_full[1:],list_[8:]))
dict_path

{'./A_full\\20210201': ['Nintendo Switch『モンスターハンターライズ』の最新映像が公開。 _ トピックス _ Nintendo.html',
  'The savings are clear.eml',
  '【Nintendo Switch】期間限定セール・新作ほか、ソフトのご紹介.eml'],
 './A_full\\20210202': ['Your question of the week from Vocabulary.com.eml',
  '東京2020大会：オリンピック＆パラリンピックの世界へ.eml'],
 './A_full\\20210203': ['新年のご挨拶.eml', 'もっちもっちの弾力に、脈打つ熟成香。魅惑のブランデーあんぱん.eml'],
 './A_full\\20210204': ['ソフトウェアを安全に保護するためのより良い方法.eml',
  'ゴーストライター騒動から7年 新垣隆の「覚悟」.eml',
  'Communication trends of 2020.eml'],
 './A_full\\20210205': ['東京2020大会：オリンピック＆パラリンピックの世界を知り尽くそう.eml',
  '宇賀なつみ、思わず息をのむ.eml'],
 './A_full\\20210206': ['深度学习之路（一）：用LSTM网络做时间序列数据预测 - 简书.html'],
 './A_full\\20210207': ['Our cool-productivity-killer newsletter.eml'],
 './A_full\\20210208': ['明智光秀、最後の夜は…….eml']}

In [29]:
save_dir='./new/'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

In [30]:
for i in kk_full[1:]:
#     print(i.split('/')[1].split('\\')[1])
    for m in dict_path[i]:
        from_path = os.path.join(i,m)
#         print(from_path)
        to_path = os.path.join(save_dir,i.split('/')[1].split('\\')[1])
        if not os.path.isdir(to_path):
            os.makedirs(to_path)
        copy(from_path,to_path)

## 以下参考まで：

#### 判斷文件大小

In [ ]:
def get_FileSize(path):
    fsize = round(os.path.getsize(path)/float(1024*1024),2)
    
    return fsize

#### html

In [ ]:
def html_text(list_):
    text = []
    ii = []
    for i in list_:
        if i.split('.')[-1] == 'html':
            ii.append(i)
            htmlfile = open(i, 'r', encoding='utf-8-sig')
            htmlhandle = htmlfile.read()
            soup = BeautifulSoup(htmlhandle, 'html.parser')
            text.append(soup.text)
            dict_ = dict(zip(ii,text))
    return dict_

In [ ]:
path_copy= []
for i in range(len(list_[4:])):
    for m in range(len(list_[4:][i])):
        path_tmp = os.path.join(kk[i+1],list_[4:][i][m])
        path_copy.append(path_tmp)

In [ ]:
path_copy[0].split('data')

In [ ]:
folder = [path_copy[i].split('/')[2].split('\\')[0:2] for i in range(len(path_copy))]

In [ ]:
folder_path=list(set([os.path.join(i[0],i[1]) for i in folder]))

In [ ]:
folder_path

In [ ]:
save_dir='./new/'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

In [ ]:
i=0
k=0
for i in path_copy:
    for m in foler_path:
        if m in i:
            to_path = save_dir+m
            if not os.path.isdir(to_path):
                os.makedirs(to_path)
                copy(i,to_path)

In [ ]:
path=[kk[1]+'/'+list_[i] for i in range(len(list_))]

In [ ]:
# dict_tt={}
# for i in tt[1:]:
dict_tt = dict(zip(ww[0],tt[1:]))
dict_tt_copy = dict(zip(ww_copy[0],tt_copy[1:]))
#         print(m[0])
#         dict_tt= zip(m[0],i)
# dict_tt_copy={}
# for m in ww_copy[0]:
#     for i in tt_copy[1:]:
#         dict_tt_copy[m]=i

In [ ]:
path_full = [kk[i]+'/'+ tt[i][m] for i in range(1,len(kk)) for m in range(len(tt[i]))] 

In [ ]:
path_error = [kk_copy[i]+'/'+ tt_copy[i][m] for i in range(1,len(kk_copy)) for m in range(len(tt_copy[i]))] 

In [ ]:
list_=[]
for i in tt[1]:
    if i not in tt[2]:
        list_.append(i)

In [ ]:
path=[kk[1]+'/'+list_[i] for i in range(len(list_))]

In [ ]:
save_dir='./tmp/mm/new'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

In [ ]:
i=0
k=0
for i in path:
    to_path=save_dir
    print(i)
    copy(i,to_path)